In [1]:
class IntegerValue:
    def __set__(self, instance, value):
        print("__set__ called")

    def __get__(self, instance, owner_class):
        print("__get__ called")


class Point:
    x = IntegerValue()


In [2]:
p = Point()
p.x = 10
p.x

__set__ called
__get__ called


In [3]:
p.__dict__

{}

In [4]:
p.__dict__["x"] = "hello"
p.__dict__

{'x': 'hello'}

In [5]:
p.x  # Python ignores instance.__dict__ and uses data descriptor instead

__get__ called


In [6]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        print("__get__ called")


class Logger:
    current_time = TimeUTC()

In [7]:
l = Logger()
l.current_time

__get__ called


In [8]:
l.__dict__["current_time"] = "the time"
l.__dict__

{'current_time': 'the time'}

In [9]:
l.current_time  # non data descriptor not used if instance.__dict__ has same key

'the time'

In [10]:
class ValidString:

    def __init__(self, min_length = None):
        self.min_length = min_length

    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError(f"{self.property_name} must be a string")

        if self.min_length and len(value) < self.min_length:
            raise ValueError(f"{self.property_name} length must be more than {self.min_length} chars")

        # access __dict__ directly to avoid infinity loop
        # setattr(instance, self.property_name, value)
        instance.__dict__[self.property_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self

        # access __dict__ directly to avoid infinity loop
        # return getattr(instance, self.property_name)
        return instance.__dict__.get(self.property_name)



In [11]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)


p = Person()

In [12]:
p.first_name = "Alex"
p.last_name = "Smith"

In [13]:
p.__dict__

{'first_name': 'Alex', 'last_name': 'Smith'}

In [14]:
p.first_name, p.last_name

('Alex', 'Smith')

In [15]:
try:
    p.first_name = ""
except ValueError as e:
    print(e)

first_name length must be more than 1 chars
